In [ ]:
"""
Abstract Art Generator - Interactive Notebook

This notebook demonstrates how to use the art_generator module
to create abstract art using neural networks.

For more details, see the module documentation: art_generator.py
"""

import torch
import torch.nn as nn
import matplotlib.pyplot as plt
from art_generator import (
    ArtNetwork,
    create_and_generate,
    generate_image,
    plot_image,
    save_image
)

# Check for GPU availability
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

In [ ]:
# Quick Start: Generate a single image
# All the functions are now in the art_generator module

print("Generating your first abstract art piece...")
print("This may take a few seconds...")

network, image = create_and_generate(
    width=128,
    height=128,
    num_neurons=32,
    save=False,  # Set to True to save
    show=True,
    device=device
)

print("Done! The neural network has painted your abstract art.")

In [ ]:
# Generate a higher resolution image using the same network
print("Generating high-resolution version...")

high_res_image = generate_image(network, width=1080, height=720, device=device)
plot_image(high_res_image, fig_size=8)

# Optionally save it
# save_image(high_res_image, "my_art_highres.png")

In [ ]:
# Experiment: Try different activation functions

print("Comparing different activation functions...\n")

activations = {
    "Tanh": nn.Tanh,
    "ReLU": nn.ReLU,
    "LeakyReLU": nn.LeakyReLU,
    "ELU": nn.ELU,
}

for name, activation in activations.items():
    print(f"Generating with {name}...")
    _, _ = create_and_generate(
        width=128,
        height=128,
        num_neurons=32,
        activation=activation,
        save=False,
        show=True,
        device=device
    )

Let's see how the images change if we increase the depth

In [ ]:
# Experiment: Effect of network depth

print("Exploring how network depth affects the art...\n")

for num_layers in range(2, 30, 3):
    print(f"Generating with {num_layers} layers...")
    _, _ = create_and_generate(
        width=128,
        height=128,
        num_layers=num_layers,
        save=False,
        show=True,
        device=device
    )

And also the effect of increasing the width

In [ ]:
# Experiment: Effect of network width

print("Exploring how network width affects the art...\n")

for i in range(1, 10, 2):
    num_neurons = 2 ** i
    print(f"Generating with {num_neurons} neurons...")
    _, _ = create_and_generate(
        width=128,
        height=128,
        num_neurons=num_neurons,
        save=False,
        show=True,
        device=device
    )

What happens if we use ReLUs?

In [ ]:
# What happens if we use ReLUs?

print("Testing ReLU activation...")
network_relu, image_relu = create_and_generate(
    width=128,
    height=128,
    activation=nn.ReLU,
    save=False,
    show=True,
    device=device
)

print("\nReLU creates interesting sharp edges and solid color regions!")